In [16]:
!pip install transformers datasets peft accelerate bitsandbytes

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [20]:
from peft import LoraConfig, get_peft_model, TaskType
from transformers import AutoTokenizer, AutoModelForCausalLM
from transformers import Trainer, TrainingArguments, DataCollatorForLanguageModeling
from datasets import load_dataset
from huggingface_hub import login
import torch

In [18]:
login(token='NEED TOKEN')

model_name = "mistralai/Mistral-7B-v0.1"
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token

In [21]:
device = torch.device("cuda:0")

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    load_in_4bit=True
).to(device)

peft_config = LoraConfig(
    r=8,
    lora_alpha=16,
    lora_dropout=0.1,
    task_type = TaskType.CAUSAL_LM
)

model = get_peft_model(model, peft_config)

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [22]:
dataset = load_dataset("tatsu-lab/alpaca")["train"].select(range(1000))

dataset

Dataset({
    features: ['instruction', 'input', 'output', 'text'],
    num_rows: 1000
})

In [23]:
dataset[0]

{'instruction': 'Give three tips for staying healthy.',
 'input': '',
 'output': '1.Eat a balanced diet and make sure to include plenty of fruits and vegetables. \n2. Exercise regularly to keep your body active and strong. \n3. Get enough sleep and maintain a consistent sleep schedule.',
 'text': 'Below is an instruction that describes a task. Write a response that appropriately completes the request.\n\n### Instruction:\nGive three tips for staying healthy.\n\n### Response:\n1.Eat a balanced diet and make sure to include plenty of fruits and vegetables. \n2. Exercise regularly to keep your body active and strong. \n3. Get enough sleep and maintain a consistent sleep schedule.'}

In [24]:
def preprocess(ex):
    prompt = f"### Instruction:\n{ex['instruction']}\n\n### Response:\n{ex['output']}"
    tokenized = tokenizer(prompt, truncation=True, padding='max_length', max_length=512)
    tokenized['labels'] = tokenized['input_ids'].copy()
    return tokenized

tokenized_dataset = dataset.map(preprocess)

In [26]:
trainer = Trainer(
    model=model,
    args=TrainingArguments(
        per_device_train_batch_size=1,
        num_train_epochs=1,
        learning_rate=2e-4,
        output_dir="./lora-mistral-alpaca",
        fp16=True,
        logging_steps=10,
        save_steps=100,
        save_total_limit=2,
        gradient_accumulation_steps=4,
        report_to='none'
    ),
    train_dataset=tokenized_dataset,
    data_collator=DataCollatorForLanguageModeling(tokenizer, mlm=False)
)

trainer.train()

No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


Step,Training Loss
10,0.789600
20,0.704500
30,0.607100
40,0.726900
50,0.517500
60,0.702800
70,0.530800
80,0.561800
90,0.652500
100,0.608100


TrainOutput(global_step=125, training_loss=0.6229800281524658, metrics={'train_runtime': 2714.5036, 'train_samples_per_second': 0.368, 'train_steps_per_second': 0.046, 'total_flos': 2.1854416797696e+16, 'train_loss': 0.6229800281524658, 'epoch': 1.0})

In [31]:
model.save_pretrained("./lora-mistral-alpaca")

In [32]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import PeftModel

login(token='NEED TOKEN')

base_model_name = "mistralai/Mistral-7B-v0.1"  
lora_model_path = "./lora-mistral-alpaca"  # путь к LoRA

tokenizer = AutoTokenizer.from_pretrained(base_model_name)
base_model = AutoModelForCausalLM.from_pretrained(base_model_name, device_map="auto", load_in_4bit=True)
model = PeftModel.from_pretrained(base_model, lora_model_path)
model.eval()


The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): MistralForCausalLM(
      (model): MistralModel(
        (embed_tokens): Embedding(32000, 4096)
        (layers): ModuleList(
          (0-31): 32 x MistralDecoderLayer(
            (self_attn): MistralAttention(
              (q_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=4096, out_features=4096, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.1, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4096, out_features=8, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=8, out_features=4096, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              (k_proj):

In [34]:
from transformers import pipeline
pipe = pipeline("text-generation", model=model, tokenizer=tokenizer)
prompt = "### Instruction:\nExplain the difference between supervised and unsupervised learning.\n\n### Response:\n"
outputs = pipe(prompt, max_new_tokens=200, do_sample=True, temperature=0.7)
print(outputs[0]["generated_text"])


Device set to use cuda:0
The model 'PeftModelForCausalLM' is not supported for text-generation. Supported models are ['AriaTextForCausalLM', 'BambaForCausalLM', 'BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'LlamaForCausalLM', 'CodeGenForCausalLM', 'CohereForCausalLM', 'Cohere2ForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'DbrxForCausalLM', 'DeepseekV3ForCausalLM', 'DiffLlamaForCausalLM', 'ElectraForCausalLM', 'Emu3ForCausalLM', 'ErnieForCausalLM', 'FalconForCausalLM', 'FalconMambaForCausalLM', 'FuyuForCausalLM', 'GemmaForCausalLM', 'Gemma2ForCausalLM', 'Gemma3ForConditionalGeneration', 'Gemma3ForCausalLM', 'GitForCausalLM', 'GlmForCausalLM', 'Glm4ForCausalLM', 'GotOcr2ForConditionalGeneration', 'GPT2LMHeadModel', 'GPT2LMHeadModel', 'GPTBigCodeForCausalLM', 'GPTNeoFo

### Instruction:
Explain the difference between supervised and unsupervised learning.

### Response:
Supervised learning is a type of machine learning that uses labeled datasets to train models. Unsupervised learning, on the other hand, uses unlabeled data to extract patterns and features from data. Supervised learning is typically used to solve specific, known problems, while unsupervised learning is used to discover hidden patterns in data. Unsupervised learning is used when there is not enough information to classify data into predefined categories, and is often used in exploratory analysis of data.
